In [2]:
#importamos librerías y herramientas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, OrdinalEncoder

import datetime as dt
from datetime import datetime
from dateutil.relativedelta import relativedelta
# gráficos
import seaborn as sns
import matplotlib.pyplot as plt
# preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
 # modelos
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
#from xgboost import XGBRegressor
from catboost import CatBoostRegressor
# metricas
import sklearn.metrics as metrics
# hiperparametrizado
from sklearn.model_selection import GridSearchCV
# quitar alertas
import warnings
warnings.filterwarnings("ignore")

In [3]:
df_entrenar = pd.read_csv('data/viviendas_regresion_basica.csv')